In [1]:
import torch
from torch import nn,optim
import numpy as np
import torchvision 
from torchvision import transforms

In [55]:
batch_size=256
mnist_train=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=True,
    download=True,transform=transforms.ToTensor())
mnist_test=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=False,
    download=True,transform=transforms.ToTensor())

In [56]:
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size,shuffle=True)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size,shuffle=True)

In [57]:
num_inputs,num_outputs,num_hiddens=784,10,256
w1=torch.Tensor(np.random.normal(0,0.01,(num_inputs,num_hiddens)))
b1=torch.zeros(num_hiddens,dtype=torch.float32)
w2=torch.Tensor(np.random.normal(0,0.01,(num_hiddens,num_outputs)))
b2=torch.zeros(num_outputs,dtype=torch.float32)

params=[w1,b1,w2,b2]
for p in params:
    p.requires_grad_(requires_grad=True)


In [58]:
def relu(X):
    return torch.max(input=X,other=torch.Tensor([0.0]))

In [59]:
def net(X):
    X=X.view((-1,num_inputs))
    H=relu(torch.matmul(X,w1)+b1)
    return torch.matmul(H,w2)+b2

In [61]:
def sgd(params,lr,batch_size):
    for p in params:
        p.data -= lr * p.grad/batch_size

In [62]:
loss=nn.CrossEntropyLoss()

In [78]:
def eval_acc(data_iter):
    acc,n=0,0
    for x,y in data_iter:
        acc+=(net(x).argmax(dim=1)==y).float().sum().item()
        n+=len(y)
# #         print(net(x).argmax(dim=1))
# #         print(y)
#         print('-----------------------')
#         print(acc)
#     print('test acc:',acc/y)
    return acc/n

In [80]:
lr=100
for i in range(5):
    train_l,train_acc,test_acc,n=0,0,0,0
    for x,y in train_iter:
        y_hat=net(x)
        l=loss(y_hat,y).sum()
        
        if params[0].grad is not None:
            for p in params:
                p.grad.data.zero_()
        l.backward()
        sgd(params,lr,batch_size)
        train_l+=l.item()
        train_acc+=(y_hat.argmax(dim=1)==y).float().sum().item()
        n+=len(y)
    test_acc=eval_acc(test_iter)
    print('epoch %d,train loss : %.3f,train acc %.3f ,test acc: %.3f' % (
    i+1,
        train_l/235,
        train_acc/n,
        test_acc))
        

epoch 1,train loss : 0.766,train acc 0.719 ,test acc: 0.745
epoch 2,train loss : 0.481,train acc 0.824 ,test acc: 0.826
epoch 3,train loss : 0.426,train acc 0.844 ,test acc: 0.829
epoch 4,train loss : 0.396,train acc 0.855 ,test acc: 0.837
epoch 5,train loss : 0.371,train acc 0.863 ,test acc: 0.817
